In [1]:
import sys
import pickle
import glob
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import warnings

warnings.filterwarnings(action="once")

import BioSimSpace as BSS

import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *

BSS.__file__

import py3Dmol as _py3Dmol
from rdkit import Chem as _Chem
import tempfile as _tempfile

INFO:rdkit:Enabling RDKit 2024.03.5 jupyter extensions
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/site-packages/gufe/settings/models.py:42: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  extra = pydantic.Extra.forbid
<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:671: ImportWarning: _SixMetaPathImporter.exec_module() not found; falling back to load_module()
<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MA

In [2]:
def draw_labelled_molecule(lig_0):
    # Create a temporary working directory and store the directory name.
    tmp_dir = _tempfile.TemporaryDirectory()
    work_dir = tmp_dir.name

    BSS.IO.saveMolecules(work_dir + "/molecule0", lig_0, "PDB")
    # Load the molecules into RDKit.
    rdmol0 = _Chem.MolFromPDBFile(
        work_dir + "/molecule0.pdb", sanitize=False, removeHs=False
    )

    # Set grid view properties.
    pixels = 900
    width = pixels / 2
    height = pixels

    # Create the view.
    view = _py3Dmol.view(linked=False, width=width, height=height)
    style = {"stick": {"colorscheme": "grayCarbon", "linewidth": 0.1}}

    # Add the molecules to the views.
    view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0")

    # Set the style.
    view.setStyle({"model": 0}, style)

    # Highlight the atoms from the mapping.
    for atom0 in lig_0.getAtoms():
        p = rdmol0.GetConformer().GetAtomPosition(atom0.index())
        # p = rdmol0.GetAtomWithIdx(atom0.index())
        view.addSphere(
            {
                "center": {"x": p.x, "y": p.y, "z": p.z},
                "radius": 0.5,
                "color": "green",
                "alpha": 0.8,
            },
        )
        view.addLabel(f"{atom0.index()}", {"position": {"x": p.x, "y": p.y, "z": p.z}})

        # Set background colour.
        view.setBackgroundColor("white")

        # Zoom to molecule.
        view.zoomTo()

    return view


def view_mapping(lig_0, lig_1, mapping):
    # Write the molecules to PDB format.
    BSS.IO.saveMolecules("molecule0", lig_0, "PDB")
    BSS.IO.saveMolecules("molecule1", lig_1, "PDB")

    # Load the molecules into RDKit.
    rdmol0 = _Chem.MolFromPDBFile("molecule0.pdb", sanitize=False, removeHs=False)
    rdmol1 = _Chem.MolFromPDBFile("molecule1.pdb", sanitize=False, removeHs=False)

    # mapping = BSS.Align.matchAtoms(
    #                         lig_0, lig_1
    #                         )

    # Set grid view properties.
    viewer0 = (0, 0)
    pixels = 900
    viewergrid = (2, 1)
    viewer1 = (1, 0)
    width = pixels / 2
    height = pixels

    # Create the view.
    view = _py3Dmol.view(
        linked=False, width=width, height=height, viewergrid=viewergrid
    )
    style = {"stick": {"colorscheme": "grayCarbon", "linewidth": 0.1}}

    # Add the molecules to the views.
    view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0", viewer=viewer0)
    view.addModel(_Chem.MolToMolBlock(rdmol1), "mol1", viewer=viewer1)

    # Set the style.
    view.setStyle({"model": 0}, style, viewer=viewer0)
    view.setStyle({"model": 0}, style, viewer=viewer1)

    # Highlight the atoms from the mapping.
    for atom0, atom1 in mapping.items():
        p = rdmol0.GetConformer().GetAtomPosition(atom0)
        view.addSphere(
            {
                "center": {"x": p.x, "y": p.y, "z": p.z},
                "radius": 0.5,
                "color": "green",
                "alpha": 0.8,
            },
            viewer=viewer0,
        )
        view.addLabel(
            f"{atom0} \u2192 {atom1}",
            {"position": {"x": p.x, "y": p.y, "z": p.z}},
            viewer=viewer0,
        )
        p = rdmol1.GetConformer().GetAtomPosition(atom1)
        view.addSphere(
            {
                "center": {"x": p.x, "y": p.y, "z": p.z},
                "radius": 0.5,
                "color": "green",
                "alpha": 0.8,
            },
            viewer=viewer1,
        )
        view.addLabel(
            f"{atom1} \u2192 {atom0}",
            {"position": {"x": p.x, "y": p.y, "z": p.z}},
            viewer=viewer1,
        )

    # Set background colour.
    view.setBackgroundColor("white", viewer=viewer0)
    view.setBackgroundColor("white", viewer=viewer1)

    # Zoom to molecule.
    view.zoomTo(viewer=viewer0)
    view.zoomTo(viewer=viewer1)

    return view

In [3]:
lig1 = "lig_2n"
lig2 = "lig_2o"
prot = "p38"
prep_dir = (
    f"/home/anna/Documents/benchmark/{prot}_benchmark/prep"  # /backup/{prot}/prep
)
name = "sys"

system_1 = BSS.IO.readMolecules(
    [
        f"{prep_dir}/{lig1}_{name}_equil_solv.rst7",
        f"{prep_dir}/{lig1}_{name}_equil_solv.prm7",
    ]
)
system_2 = BSS.IO.readMolecules(
    [
        f"{prep_dir}/{lig2}_{name}_equil_solv.rst7",
        f"{prep_dir}/{lig2}_{name}_equil_solv.prm7",
    ]
)

lig_0 = merge.extract_ligand(system_1)
lig_1 = merge.extract_ligand(system_2)

import pickle

try:
    with open(
        f"/home/anna/Documents/benchmark/inputs/mapping/{lig1}~{lig2}_mapping_dict.pickle",
        "rb",
    ) as file:
        prematch = pickle.load(file)
        print("is prematch")
except:
    prematch = None

kwargs = {
    "prematch": prematch,
    # "prematch": {8: 10},  # 30:27, 34:31, 41:38, 4:4
    # "allow ring size change": False,
    # "allow ring breaking": False,
    # "complete rings only": True,
    # "ring matches ring only": True,
    # "match chiral tag": False,
    # "natch valence": False,
    # "maximise bonds": False,
    # "PRUNEPERTURBEDCONSTRAINTS": False,
    # "prune crossing constraints": False,
    "scoring function": "rmsd_flex_align",
}

/home/anna/BioSimSpace/python/BioSimSpace/IO/_io.py:459: UserWarning: BioSimSpace.IO: Please install GROMACS (http://www.gromacs.org) for GROMACS topology file support.
  _warnings.warn(
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp33jzqsrz'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpukdx_wwm'>
  _warnings.warn(warn_message, ResourceWarning)


In [12]:
lig1 = "lig_ejm50"
lig2 = "lig_ejm54"
prot = "tyk2"

# prep_dir = f"/home/anna/Documents/benchmark"

# lig_0 = pipeline.prep.ligprep.lig_paramaterise(
#     BSS.IO.readMolecules(f"{prep_dir}/inputs/reruns/{prot}/ligands/{lig1}.sdf")[0], "gaff2"
# )
# lig_1 = pipeline.prep.ligprep.lig_paramaterise(
#     BSS.IO.readMolecules(f"{prep_dir}/inputs/reruns/{prot}/ligands/{lig2}.sdf")[0], "gaff2"
# )


prep_dir = (
    # /backup/{prot}/prep
    f"/home/anna/Documents/benchmark/reruns/{prot}/prep"
)
name = "lig"

system_1 = BSS.IO.readMolecules(
    [
        f"{prep_dir}/{lig1}_{name}_equil_solv.rst7",
        f"{prep_dir}/{lig1}_{name}_equil_solv.prm7",
    ]
)
system_2 = BSS.IO.readMolecules(
    [
        f"{prep_dir}/{lig2}_{name}_equil_solv.rst7",
        f"{prep_dir}/{lig2}_{name}_equil_solv.prm7",
    ]
)

lig_0 = merge.extract_ligand(system_1)
lig_1 = merge.extract_ligand(system_2)

/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmptwe24hc0'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmprula3s2l'>
  _warnings.warn(warn_message, ResourceWarning)


In [13]:
kwargs = {
    "prematch": {},  # 30:27, 34:31, 41:38, 4:4
    "allow ring size change": False,
    "allow ring breaking": False,
    "complete rings only": True,
    "ring matches ring only": True,
    "match chiral tag": False,
    "natch valence": False,
    "maximise bonds": False,
    "PRUNEPERTURBEDCONSTRAINTS": False,
    "prune crossing constraints": False,
    "prune atom types" : False,
    "scoring function": "rmsd_flex_align",
}

In [14]:
merge.no_perturbing_atoms_average(
    lig_0,
    lig_1,
    **kwargs
)  # **kwargs)

INFO:root:mapping atoms using rmsd_flex_align, complete rings True, prune perturbed constraints False, prune crossing constraints False and prematch {}
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpp1enk3ue'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpsc0fk96u'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpw0m1vsun'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpkfwn737_'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/en

3.0

In [15]:
l0a, l1a, mapping = pipeline.prep.merge.atom_mappings(lig_0, lig_1, **kwargs)

INFO:root:mapping atoms using rmsd_flex_align, complete rings True, prune perturbed constraints False, prune crossing constraints False and prematch {}
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp4ghzr6m7'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpkb2dcxqo'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpiv31jkrk'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/envs/pipeline_annamherz/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp84flp_ur'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/mambaforge/en

In [16]:
view = view_mapping(lig_0, lig_1, mapping)
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
# changes to mapping

del mapping[40]
# mapping[37] = 13

In [ ]:
print(mapping)

with open(
    f"/home/anna/Documents/benchmark/reruns/{prot}/inputs/mapping/{lig1}~{lig2}_mapping_dict.pickle",
    "wb",
) as handle:
    pickle.dump(mapping, handle)

In [ ]:
view = draw_labelled_molecule(lig_0)
view

In [ ]:
view = draw_labelled_molecule(lig_1)
view